In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

df_raw = pd.read_json("restaurant_reviews_1900k.json", lines=True)

# df = pd.read_csv("spacy_train_clean.csv")
# df['entities_clean'] = df['entities_clean'].apply(lambda x: ast.literal_eval(x))
# df['entities'] = df.apply(lambda x: [x['text'][i[0]:i[1]] for i in x['entities_clean']],axis=1)

# only get restaurants with many reviews
many_reviews = df_raw[['business_id','review_id']].groupby("business_id")['review_id'].nunique()
many_reviews = many_reviews[many_reviews > 1000].index
df = df_raw[df_raw.business_id.isin(set(many_reviews))]

In [67]:
print("Number of businesses in subset: ", len(df.business_id.unique()))

Number of businesses in subset:  142


In [ ]:
business_stars = df[['business_id', 'stars']].groupby('business_id').mean()
business_with_same_stars = business_stars[
    (business_stars.stars > 3.5) 
    & (business_stars.stars < 4.5)].index
print(business_with_same_stars)

In [86]:
# subset reviews
bus = df[df.business_id == '-6tvduBzjLI1ISfs3F_qTg']

In [113]:
bus.text.iloc[20]

'We ended up here after the place next door had an hour wait. They sold us on their $5 drinks but ended up staying for lunch. I got the chicken burrito which looked unassuming but was full of flavor and huge. My wife got the kale salad and it was awesome. We will definitely be back !'

In [120]:
entities = [
    'salad',
    'atmosphere', 
    'burrito',
    'food',
    'taco',
    'chicken burrito',
    'chicken taco',
    'carnitas',
    'pork adobada',
    'carne asada',
    'fish taco',
    'shrimp taco',
    'chips',
    'salsa',
    'beer',
    'tequila',
    'margarita',
    'mojito'
]

In [121]:
# show average rating for that restaurant
print("Average Ratings: ", np.mean(bus.stars))

Average Ratings:  3.8768844221105527


In [167]:
# INSERT FULL NLP process here, for each review, run NLP to create
# set of entities and NLP scores to each review

# get restaurants with enough entities that have reviews (NEED ER MODEL)

# for that restaurant, get set of entities which have enough reviews
# (usually with NER, but right now hard code entities, assume NER works well)

# manual entity num check 
# for ent in entities:
#     print("===================================\n")
#     sub = bus[bus.text.str.contains(ent, case=False)]
#     print("Entity: ", ent)
#     print("Score: ", np.mean(sub.stars))
#     print("Number of Reviews: ", len(sub))
#     print("\n")

import random
from collections import defaultdict
rankings = defaultdict(list)

# grab all reviews, average ratings 
for ent in entities:
    rankings['entity'] += [ent]
    entity_reviews = bus[bus.text.str.contains(ent, case=False)]
    rankings['average_stars'] += [np.mean(entity_reviews.stars)]
    # insert NLP process here
    rankings['predicted'] += [random.uniform(0,5)]
# for each entity, calculate avg true rating / average sentiment score (optimization: run before hand)
rankings = pd.DataFrame(rankings)

In [168]:
from scipy.stats import spearmanr
# rankings.predicted = rankings.average_stars
rankings

,entity,average_stars,predicted
0,salad,3.969072,4.694935
1,atmosphere,4.022124,3.188274
2,burrito,3.813793,1.071066
3,food,3.675159,4.760916
4,taco,3.926329,4.733622
5,chicken burrito,3.695652,1.662370
6,chicken taco,3.863636,1.428835
7,carnitas,4.135593,3.772582
8,pork adobada,4.000000,2.369075
9,carne asada,3.826590,3.080352


In [169]:
# spearman correlation metric
corr, pvalue = spearmanr(rankings.average_stars, rankings.predicted)
print("Spearman Correlation: ", corr)

Spearman Correlation:  -0.11042311661506708
